In [ ]:
# import libaries
import os
import pandas as pd
import numpy as np
import polars as pl
import mlxtend as ml

In [ ]:
# import data
df = pl.read_excel("data_seance_4/clients banque.xlsx")

In [ ]:
# head of data
print(df.head())

In [ ]:
# names of columns
print(df.columns)

In [ ]:
# create a new df without Note attribuée
df_filtred = df.drop("note attribuee")

In [ ]:
# get_dimunus
df_filtred.to_dummies()

In [ ]:
# For the original df (using Age du client as numeric column)
count_23_40_df = df.filter(pl.col("Age du client") == "de 23 a 40 ans").shape[0]

# For df_filtred (using string column)
count_23_40_df_filtred = df_filtred.filter(pl.col("Age du client") == "de 23 a 40 ans").shape[0]

print(f"Number of clients aged 23-40 in original df: {count_23_40_df}")
print(f"Number of clients aged 23-40 in filtered df: {count_23_40_df_filtred}")


### **ON S’INTERESSE AUX ITEMSETS FREQUENTS**


In [ ]:
# modifie the number of lines to display with polar
pl.Config.set_tbl_cols(50)

In [ ]:
from mlxtend.frequent_patterns import apriori

# Convert polars DataFrame to pandas DataFrame
df_pandas = df_filtred.to_pandas()

# Create boolean DataFrame (get_dummies)
df_encoded = pd.get_dummies(df_pandas)


In [19]:
# Apply apriori algorithm
frequent_itemsets = apriori(df_encoded, 
                          min_support=0.5,    # 50% support threshold
                          max_len=2,          # Maximum length of 2 items
                          use_colnames=True)

# Sort by support value in descending order
frequent_itemsets = frequent_itemsets.sort_values('support', ascending=False)

# Display results
print(frequent_itemsets)

     support                                           itemsets
5   0.880342       (Interdiction de chequier_chequier autorise)
1   0.784188         (Domiciliation de l_epargne_pas d_epargne)
0   0.670940        (Domiciliation du salaire_domicile salaire)
10  0.670940  (Domiciliation de l_epargne_pas d_epargne, Int...
3   0.655983             (Moyenne en cours_de 2 a 5 KF encours)
8   0.613248  (Domiciliation du salaire_domicile salaire, In...
9   0.600427  (Moyenne en cours_de 2 a 5 KF encours, Domicil...
11  0.576923  (Moyenne en cours_de 2 a 5 KF encours, Interdi...
4   0.559829     (Autorisation de decouvert_decouvert interdit)
12  0.521368  (Autorisation de decouvert_decouvert interdit,...
2   0.510684                               (Profession_employe)
6   0.504274                        (Type de client_bon client)
7   0.502137  (Domiciliation du salaire_domicile salaire, Do...


In [20]:
# type d'object 
print(type(frequent_itemsets))

<class 'pandas.core.frame.DataFrame'>


In [21]:
# filter the result with a itemset of 2
frequent_itemsets_2 = frequent_itemsets[frequent_itemsets['itemsets'].map(len) == 2]

In [22]:
# print the result
print(frequent_itemsets_2)

     support                                           itemsets
10  0.670940  (Domiciliation de l_epargne_pas d_epargne, Int...
8   0.613248  (Domiciliation du salaire_domicile salaire, In...
9   0.600427  (Moyenne en cours_de 2 a 5 KF encours, Domicil...
11  0.576923  (Moyenne en cours_de 2 a 5 KF encours, Interdi...
12  0.521368  (Autorisation de decouvert_decouvert interdit,...
7   0.502137  (Domiciliation du salaire_domicile salaire, Do...


In [23]:
# Calculate the support manually
total_rows = len(df_pandas)
matching_rows = df_pandas[
    (df_pandas['Autorisation de decouvert'] == 'decouvert interdit') & 
    (df_pandas['Interdiction de chequier'] == 'chequier autorise')
].shape[0]

support = matching_rows / total_rows
print(f"Manual support calculation: {support:.6f}")

# Get the support value from apriori results for comparison
apriori_support = frequent_itemsets_2[
    frequent_itemsets_2['itemsets'] == frozenset({
        'Autorisation de decouvert_decouvert interdit',
        'Interdiction de chequier_chequier autorise'
    })
]['support'].values[0]
print(f"Apriori support value: {apriori_support:.6f}")

Manual support calculation: 0.521368
Apriori support value: 0.521368


In [29]:
for items in frequent_itemsets_2["itemsets"]:
    # search for the itemset containing "Interdiction_de_chequier"
    if "Interdiction de chequier_chequier autorise" in items:
        print(items)

frozenset({'Domiciliation de l_epargne_pas d_epargne', 'Interdiction de chequier_chequier autorise'})
frozenset({'Domiciliation du salaire_domicile salaire', 'Interdiction de chequier_chequier autorise'})
frozenset({'Moyenne en cours_de 2 a 5 KF encours', 'Interdiction de chequier_chequier autorise'})
frozenset({'Autorisation de decouvert_decouvert interdit', 'Interdiction de chequier_chequier autorise'})


In [31]:
# Calculate proportions for each configuration
config_stats = []

for decouvert in df_pandas['Autorisation de decouvert'].unique():
    for chequier in df_pandas['Interdiction de chequier'].unique():
        # Filter for this configuration
        mask = (df_pandas['Autorisation de decouvert'] == decouvert) & \
               (df_pandas['Interdiction de chequier'] == chequier)
        config_df = df_pandas[mask]
        
        if len(config_df) > 0:  # Only process if configuration exists
            # Calculate proportions
            total = len(config_df)
            good_clients = (config_df['Type de client'] == 'bon client').sum() / total * 100
            bad_clients = (config_df['Type de client'] == 'mauvais client').sum() / total * 100
            
            config_stats.append({
                'Configuration': f"{decouvert} + {chequier}",
                'Total Clients': total,
                'Good Clients %': good_clients,
                'Bad Clients %': bad_clients
            })

# Convert to DataFrame and sort by percentage of good clients
result_df = pd.DataFrame(config_stats)
print("\nConfigurations sorted by percentage of good clients:")
print(result_df.sort_values('Good Clients %', ascending=False))


Configurations sorted by percentage of good clients:
                                       Configuration  Total Clients  \
0             decouvert interdit + chequier autorise            244   
2             decouvert autorise + chequier autorise            168   
3             decouvert autorise + chequier interdit             37   
1             decouvert interdit + chequier interdit             18   
4  Autorisation de decouvert + Interdiction de ch...              1   

   Good Clients %  Bad Clients %  
0       61.475410      38.524590  
2       47.619048      52.380952  
3       16.216216      83.783784  
1        0.000000     100.000000  
4        0.000000       0.000000  


In [33]:
# Calculate mean scores for each configuration
config_scores = []

for decouvert in df_pandas['Autorisation de decouvert'].unique():
    for chequier in df_pandas['Interdiction de chequier'].unique():
        # Filter for this configuration
        mask = (df['Autorisation de decouvert'] == decouvert) & \
               (df['Interdiction de chequier'] == chequier)
        config_df = df.filter(mask)
        
        if config_df.shape[0] > 0:  # Only process if configuration exists
            mean_score = config_df['note attribuee'].mean()
            total_clients = config_df.shape[0]
            
            config_scores.append({
                'Configuration': f"{decouvert} + {chequier}",
                'Total Clients': total_clients,
                'Mean Score': mean_score
            })

# Convert to DataFrame and sort by mean score
score_results = pd.DataFrame(config_scores)
print("\nConfigurations sorted by average score:")
print(score_results.sort_values('Mean Score', ascending=False))


Configurations sorted by average score:
                                       Configuration  Total Clients  \
4  Autorisation de decouvert + Interdiction de ch...              1   
0             decouvert interdit + chequier autorise            244   
2             decouvert autorise + chequier autorise            168   
3             decouvert autorise + chequier interdit             37   
1             decouvert interdit + chequier interdit             18   

   Mean Score  
4  690.000000  
0  566.004098  
2  557.559524  
3  506.351351  
1  453.055556  


### **2.2.2 Association Rules**